# Rental forecasting
- Clean rental dataframe
- Transform dataframe
- Forecast population using fb prophet
- Clean predictions

In [1]:
! pip install prophet

     |████████████████████████████████| 71kB 2.8MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 184kB 23.0MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-cp37-none-any.whl size=6639854 sha256=79b27902fb7f340e2ec9226e3ec8547bc6abef23a49e816f6482d41969e2597a
  Stored in directory: /root/.cache/pip/wheels/92/1d/02/bc0a91bd3a84e060722075d2b0b68e875e37ec7dc49451d513
Successfully built prophet
ERROR: fbprophet 0.7.1 has requirement cmdstanpy==0.9.5, but you'll have cmdstanpy 0.9.68 which is incompatible.
  Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5


In [2]:
from prophet import Prophet
import pandas as pd
import numpy as np

In [3]:
rental = pd.read_csv('https://raw.githubusercontent.com/jiobu1/labspt15-cityspire-g-ds/main/notebooks/model/rental/csv/Metro_ZORI_AllHomesPlusMultifamily_Smoothed.csv')

In [4]:
print(rental.shape)
rental.head()

(106, 89)


,RegionID,RegionName,SizeRank,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02
0,102001,United States,0,1325.0,1331.0,1338.0,1350.0,1370.0,1385,1396,1401,1402,1402.0,1401,1401.0,1406,1415,1424,1437,1447,1456,1462,1466,1468,1468,1469,1468,1473,1481,1491,1502,1512,1520,1521,1524,1524,1523,1522,1522,1525,...,1566,1567,1572,1579,1589,1599,1610,1619,1625,1627,1627,1625,1627,1627,1630.0,1638.0,1648,1659,1670,1680,1685,1687,1685,1682,1680,1684,1691,1699,1706,1702,1701,1703,1703,1695,1691,1686,1694,1693,1701,1708
1,394913,"New York, NY",1,2282.0,2300.0,2304.0,2336.0,2370.0,2392,2406,2415,2416,2415.0,2414,2413.0,2418,2430,2449,2479,2492,2510,2520,2522,2530,2530,2520,2516,2516,2520,2529,2556,2570,2582,2566,2578,2581,2576,2574,2561,2566,...,2599,2594,2585,2595,2606,2626,2641,2657,2666,2668,2669,2659,2648,2642,2645.0,2655.0,2669,2675,2705,2730,2739,2743,2734,2735,2712,2702,2704,2728,2738,2722,2712,2696,2666,2601,2551,2496,2472,2451,2472,2471
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,1735.0,1761.0,1767.0,1794.0,1832.0,1863,1888,1904,1909,1918.0,1926,1927.0,1938,1945,1961,1978,1991,2002,2018,2032,2045,2048,2060,2065,2077,2089,2102,2120,2140,2150,2153,2168,2176,2184,2182,2189,2193,...,2287,2288,2298,2308,2315,2329,2338,2357,2363,2373,2378,2389,2395,2396,2396.0,2405.0,2412,2421,2428,2443,2455,2462,2465,2462,2463,2473,2487,2492,2489,2468,2452,2458,2457,2452,2449,2451,2464,2459,2463,2472
3,394463,"Chicago, IL",3,1413.0,1423.0,1426.0,1442.0,1464.0,1480,1489,1492,1489,1482.0,1479,1476.0,1482,1487,1477,1488,1509,1526,1534,1540,1536,1528,1521,1521,1528,1536,1547,1563,1575,1586,1590,1589,1580,1569,1561,1557,1554,...,1585,1586,1590,1600,1611,1622,1636,1646,1650,1648,1638,1626,1620,1619,1627.0,1636.0,1650,1665,1678,1686,1692,1694,1685,1677,1670,1668,1676,1684,1693,1689,1690,1691,1685,1672,1654,1636,1628,1624,1622,1634
4,394514,"Dallas-Fort Worth, TX",4,1134.0,1127.0,1137.0,1154.0,1174.0,1190,1198,1203,1205,1202.0,1203,1206.0,1207,1214,1227,1237,1247,1257,1263,1267,1266,1264,1268,1262,1265,1278,1289,1301,1312,1325,1330,1333,1332,1335,1333,1332,1334,...,1379,1379,1383,1389,1395,1403,1409,1421,1425,1422,1420,1422,1431,1427,1430.0,1438.0,1443,1455,1465,1475,1479,1479,1476,1478,1475,1476,1481,1487,1491,1485,1480,1485,1491,1489,1491,1492,1505,1510,1507,1519


In [5]:
rental.isnull().sum()

RegionID      0
RegionName    0
SizeRank      0
2014-01       5
2014-02       1
             ..
2020-10       0
2020-11       0
2020-12       0
2021-01       0
2021-02       0
Length: 89, dtype: int64

In [6]:
rental.columns

Index(['RegionID', 'RegionName', 'SizeRank', '2014-01', '2014-02', '2014-03',
       '2014-04', '2014-05', '2014-06', '2014-07', '2014-08', '2014-09',
       '2014-10', '2014-11', '2014-12', '2015-01', '2015-02', '2015-03',
       '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09',
       '2015-10', '2015-11', '2015-12', '2016-01', '2016-02', '2016-03',
       '2016-04', '2016-05', '2016-06', '2016-07', '2016-08', '2016-09',
       '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03',
       '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09',
       '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03',
       '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09',
       '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03',
       '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09',
       '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03',
       '2020-04', '2020-05', '2020-06', '2020-

## Separate city state
- separate city and states
- use explode to separate combined cities and state
- combine separated city and states for dataframe grouping

In [7]:
rental[['City', 'State']] = rental['RegionName'].str.split(', ', expand=True)

In [8]:
def explode_str(population, col='City', sep='-'):
    s = population[col]
    i = np.arange(len(s)).repeat(s.str.count(sep) +1)
    return population.iloc[i].assign(**{col: sep.join(s).split(sep)})

rental = explode_str(rental)

In [9]:
rental['City'] = rental['City'].str.strip()
rental['State'] = rental['State'].str.strip()
rental['RegionName'] = rental['RegionName'].str.strip()

In [10]:
print(rental['City'].nunique())
rental['City'].unique()

115


array(['United States', 'New York', 'Los Angeles', 'Long Beach',
       'Anaheim', 'Chicago', 'Dallas', 'Fort Worth', 'Philadelphia',
       'Houston', 'Washington', 'Miami', 'Fort Lauderdale', 'Atlanta',
       'Boston', 'San Francisco', 'Detroit', 'Riverside', 'Phoenix',
       'Seattle', 'Minneapolis', 'St Paul', 'San Diego', 'St. Louis',
       'Tampa', 'Baltimore', 'Denver', 'Pittsburgh', 'Portland',
       'Charlotte', 'Sacramento', 'San Antonio', 'Orlando', 'Cincinnati',
       'Cleveland', 'Kansas City', 'Las Vegas', 'Columbus',
       'Indianapolis', 'San Jose', 'Austin', 'Virginia Beach',
       'Nashville', 'Providence', 'Milwaukee', 'Jacksonville', 'Memphis',
       'Oklahoma City', 'Louisville', 'Jefferson County', 'Hartford',
       'Richmond', 'New Orleans', 'Buffalo', 'Raleigh', 'Birmingham',
       'Salt Lake City', 'Rochester', 'Grand Rapids', 'Tucson',
       'Urban Honolulu', 'Tulsa', 'Fresno', 'Worcester', 'Stamford',
       'Albuquerque', 'Albany', 'Omaha', 'New H

In [11]:
rental['City'] = rental['City'].str.replace('Urban Honolulu', 'Honolulu')
rental['RegionName'] = rental['RegionName'].str.replace('Urban Honolulu', 'Honolulu')
rental.loc[rental['City'] == 'Honolulu']

,RegionID,RegionName,SizeRank,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,...,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,City,State
54,753924,"Honolulu, HI",54,1685.0,1637.0,1670.0,1702.0,1773.0,1884,1874,1935,1895,1954.0,1927,2036.0,2004,1946,1967,1974,1999,1994,2000,2005,2003,1995,2052,2106,2098,2043,2110,2100,2034,2054,2096,2070,2036,2084,2148,2063,2056,...,2044,2071,2101,2067,2074,2082,2095,2130,2140,2088,2065,2120,2133.0,2163.0,2125,2127,2115,2126,2134,2138,2132,2186,2182,2149,2135,2162,2139,2127,2122,2119,2098,2135,2158,2108,2160,2181,2161,2238,Honolulu,HI


In [12]:
print(rental['State'].nunique())
rental['State'].unique()

40


array([None, 'NY', 'CA', 'IL', 'TX', 'PA', 'DC', 'FL', 'GA', 'MA', 'MI',
       'AZ', 'WA', 'MN', 'MO', 'MD', 'CO', 'OR', 'NC', 'OH', 'NV', 'IN',
       'VA', 'TN', 'RI', 'WI', 'OK', 'KY', 'CT', 'LA', 'AL', 'UT', 'HI',
       'NM', 'NE', 'SC', 'AR', 'KS', 'ID', 'IA', 'MS'], dtype=object)

In [13]:
# Combine cleaned city, states back togethr
rental['RegionName'] = rental['City'] + ', ' + rental['State']

In [14]:
rental = rental[rental['RegionID'] != 102001]

In [15]:
rental = rental.drop(columns=['SizeRank', 'City', 'State'])

In [16]:
rental.to_csv('rental_cleaned.csv', index=False)

## Stack DataFrame
- create ds and y columns
(needed to run prophet

In [17]:
# to get in the form prophet needs
rent_melt = (rental.melt(id_vars=['RegionID','RegionName'], 
                         var_name='ds', 
                         value_name='y')
                         ).reset_index(drop=True)

In [18]:
rent_melt.head()

,RegionID,RegionName,ds,y
0,394913,"New York, NY",2014-01,2282.0
1,753899,"Los Angeles, CA",2014-01,1735.0
2,753899,"Long Beach, CA",2014-01,1735.0
3,753899,"Anaheim, CA",2014-01,1735.0
4,394463,"Chicago, IL",2014-01,1413.0


In [19]:
# create city list
cities_list = list(rent_melt['RegionName'].unique())

In [20]:
len(cities_list)

114

In [21]:
def rnd_series(city):
    subset = rent_melt[rent_melt['RegionName']==city]
    dates =(pd.DataFrame({'ds':pd.to_datetime(rent_melt['ds'])}))
    return subset

In [22]:
series = [rnd_series(city) for city in cities_list]

In [23]:
series[0]

,RegionID,RegionName,ds,y
0,394913,"New York, NY",2014-01,2282.0
114,394913,"New York, NY",2014-02,2300.0
228,394913,"New York, NY",2014-03,2304.0
342,394913,"New York, NY",2014-04,2336.0
456,394913,"New York, NY",2014-05,2370.0
...,...,...,...,...
9234,394913,"New York, NY",2020-10,2496.0
9348,394913,"New York, NY",2020-11,2472.0
9462,394913,"New York, NY",2020-12,2451.0
9576,394913,"New York, NY",2021-01,2472.0


In [24]:
def run_prophet(series):
    model = Prophet()
    model.fit(series)
    forecast = model.make_future_dataframe(periods=12, freq='MS')
    forecast = model.predict(forecast)
    forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast['RegionName'] = series['RegionName'].iloc[0]
    forecast = forecast[['RegionName','ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].astype(float)
    forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].apply(np.ceil)
    return forecast

In [25]:
f = run_prophet(series[0])
f.head()

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,RegionName,ds,yhat,yhat_lower,yhat_upper
0,"New York, NY",2014-01-01,2291.0,2259.0,2324.0
1,"New York, NY",2014-02-01,2303.0,2271.0,2337.0
2,"New York, NY",2014-03-01,2326.0,2294.0,2357.0
3,"New York, NY",2014-04-01,2352.0,2319.0,2386.0
4,"New York, NY",2014-05-01,2376.0,2345.0,2410.0


In [26]:
for i in range(len(series)):
    f = run_prophet(series[i])
    f.to_csv('rental_predictions.csv', mode='a', index='False')

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to overr

In [28]:
rental_prediction = pd.read_csv('rental_predictions.csv')

In [29]:
rental_prediction = rental_prediction.loc[rental_prediction['RegionName'] != 'RegionName']

In [30]:
rent_long = rental_prediction[['RegionName', 'ds', 'yhat']]

In [31]:
rent_long.dtypes

RegionName    object
ds            object
yhat          object
dtype: object

In [32]:
rent_long['yhat'] = pd.to_numeric(rent_long['yhat'],errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
rent_long.head()

,RegionName,ds,yhat
0,"New York, NY",2014-01-01,2291.0
1,"New York, NY",2014-02-01,2303.0
2,"New York, NY",2014-03-01,2326.0
3,"New York, NY",2014-04-01,2352.0
4,"New York, NY",2014-05-01,2376.0


In [34]:
rent_long = rent_long.pivot_table(index = 'RegionName',
                                  columns = 'ds',
                                  values = 'yhat')

In [35]:
rent_long

ds,2014-01-01,2014-02-01,2014-03-01,2014-04-01,2014-05-01,2014-06-01,2014-07-01,2014-08-01,2014-09-01,2014-10-01,2014-11-01,2014-12-01,2015-01-01,2015-02-01,2015-03-01,2015-04-01,2015-05-01,2015-06-01,2015-07-01,2015-08-01,2015-09-01,2015-10-01,2015-11-01,2015-12-01,2016-01-01,2016-02-01,2016-03-01,2016-04-01,2016-05-01,2016-06-01,2016-07-01,2016-08-01,2016-09-01,2016-10-01,2016-11-01,2016-12-01,2017-01-01,2017-02-01,2017-03-01,2017-04-01,...,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01
RegionName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Akron, OH",740.0,742.0,748.0,757.0,764.0,773.0,776.0,780.0,780.0,782.0,785.0,785.0,792.0,791.0,793.0,800.0,802.0,810.0,804.0,805.0,800.0,803.0,809.0,809.0,812.0,810.0,818.0,818.0,823.0,823.0,831.0,829.0,826.0,826.0,823.0,824.0,838.0,836.0,838.0,840.0,...,862.0,861.0,867.0,865.0,867.0,873.0,875.0,884.0,881.0,884.0,882.0,887.0,895.0,897.0,903.0,904.0,913.0,915.0,923.0,925.0,935.0,936.0,935.0,937.0,937.0,940.0,957.0,957.0,961.0,966.0,972.0,977.0,983.0,984.0,983.0,986.0,988.0,991.0,1004.0,1005.0
"Albany, NY",1034.0,1041.0,1047.0,1064.0,1068.0,1073.0,1079.0,1082.0,1091.0,1099.0,1099.0,1097.0,1104.0,1110.0,1108.0,1114.0,1116.0,1117.0,1119.0,1118.0,1117.0,1127.0,1124.0,1125.0,1131.0,1142.0,1133.0,1157.0,1147.0,1144.0,1140.0,1138.0,1149.0,1155.0,1159.0,1152.0,1160.0,1157.0,1161.0,1178.0,...,1208.0,1206.0,1213.0,1219.0,1218.0,1224.0,1226.0,1228.0,1230.0,1229.0,1229.0,1239.0,1237.0,1239.0,1245.0,1257.0,1248.0,1273.0,1263.0,1261.0,1258.0,1257.0,1268.0,1275.0,1279.0,1272.0,1282.0,1279.0,1284.0,1302.0,1297.0,1296.0,1295.0,1294.0,1301.0,1309.0,1311.0,1307.0,1316.0,1318.0
"Albuquerque, NM",957.0,963.0,961.0,966.0,973.0,982.0,987.0,991.0,989.0,987.0,986.0,986.0,988.0,993.0,990.0,997.0,1006.0,1012.0,1015.0,1021.0,1016.0,1015.0,1012.0,1014.0,1015.0,1017.0,1020.0,1021.0,1023.0,1034.0,1044.0,1041.0,1042.0,1038.0,1038.0,1033.0,1035.0,1043.0,1041.0,1044.0,...,1092.0,1092.0,1095.0,1102.0,1100.0,1108.0,1119.0,1127.0,1132.0,1140.0,1139.0,1141.0,1143.0,1149.0,1153.0,1160.0,1166.0,1171.0,1177.0,1192.0,1205.0,1207.0,1212.0,1211.0,1216.0,1214.0,1220.0,1232.0,1234.0,1241.0,1249.0,1263.0,1274.0,1278.0,1281.0,1282.0,1285.0,1286.0,1292.0,1302.0
"Allentown, PA",1085.0,1095.0,1097.0,1108.0,1118.0,1123.0,1129.0,1131.0,1136.0,1136.0,1138.0,1139.0,1139.0,1144.0,1145.0,1156.0,1162.0,1165.0,1170.0,1166.0,1171.0,1169.0,1173.0,1174.0,1176.0,1176.0,1183.0,1186.0,1198.0,1199.0,1201.0,1205.0,1207.0,1208.0,1205.0,1206.0,1205.0,1218.0,1216.0,1223.0,...,1282.0,1283.0,1284.0,1289.0,1291.0,1302.0,1308.0,1312.0,1317.0,1314.0,1322.0,1324.0,1331.0,1335.0,1340.0,1344.0,1354.0,1361.0,1376.0,1380.0,1385.0,1393.0,1398.0,1403.0,1404.0,1408.0,1410.0,1426.0,1428.0,1437.0,1451.0,1456.0,1462.0,1467.0,1473.0,1477.0,1480.0,1484.0,1487.0,1499.0
"Anaheim, CA",1735.0,1761.0,1779.0,1805.0,1830.0,1858.0,1876.0,1892.0,1906.0,1914.0,1924.0,1932.0,1945.0,1959.0,1958.0,1977.0,1992.0,2008.0,2023.0,2033.0,2041.0,2047.0,2057.0,2068.0,2085.0,2099.0,2110.0,2117.0,2127.0,2143.0,2152.0,2165.0,2172.0,2183.0,2191.0,2193.0,2198.0,2210.0,2225.0,2233.0,...,2389.0,2393.0,2404.0,2415.0,2411.0,2426.0,2438.0,2451.0,2462.0,2469.0,2466.0,2461.0,2459.0,2459.0,2464.0,2467.0,2467.0,2463.0,2462.0,2467.0,2467.0,2469.0,2467.0,2469.0,2467.0,2460.0,2456.0,2458.0,2464.0,2463.0,2464.0,2469.0,2470.0,2472.0,2469.0,2468.0,2467.0,2461.0,2461.0,2463.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [36]:
rent_long.to_csv('rent_long.csv', index=False)